# Simpsons Characters Classifier
Developed by jhonnatan.torres.suarez@gmail.com

* Using a dataset available in Kaggle
* In some frames of the training dataset there were two or three characters
* Model Weights can be downloaded from https://www.kaggle.com/jhotor/simpsons
* This notebook can be used in the Kaggle kernel
***Note*** English is not my primary language, my apologies in advance for any grammar mistake or typo

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
#Kaggle file system
#/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset/simpsons_dataset/

In [3]:
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, MaxPooling2D, Conv2D, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Model, Sequential
from keras.optimizers import RMSprop

Using data augmentation

In [4]:
IMG_SIZE = 224

TRAINING_DIR = "/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset/simpsons_dataset/"

training_datagen = ImageDataGenerator(rescale=1./255, featurewise_center=False, # set input mean to 0 over the dataset
 samplewise_center=False, # set each sample mean to 0
 featurewise_std_normalization=False, # divide inputs by std 
 samplewise_std_normalization=False, # divide each input by its std
 rotation_range=0, # randomly rotate images in the range 
 width_shift_range=0.1, # randomly shift images horizontally 
 height_shift_range=0.1, # randomly shift images vertically 
 horizontal_flip=True, # randomly flip images
 vertical_flip=False)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                                       target_size=(IMG_SIZE,IMG_SIZE) ,class_mode='categorical',
                                                       batch_size=64, 
                                                       shuffle=True)

Found 20933 images belonging to 42 classes.


There are **42** characters in the training data

In [5]:
class_map = train_generator.class_indices
inv_class_map = {v: k for k, v in class_map.items()}

A directory to be used in the "translation" of the prediction

In [6]:
inv_class_map

{0: 'abraham_grampa_simpson',
 1: 'agnes_skinner',
 2: 'apu_nahasapeemapetilon',
 3: 'barney_gumble',
 4: 'bart_simpson',
 5: 'carl_carlson',
 6: 'charles_montgomery_burns',
 7: 'chief_wiggum',
 8: 'cletus_spuckler',
 9: 'comic_book_guy',
 10: 'disco_stu',
 11: 'edna_krabappel',
 12: 'fat_tony',
 13: 'gil',
 14: 'groundskeeper_willie',
 15: 'homer_simpson',
 16: 'kent_brockman',
 17: 'krusty_the_clown',
 18: 'lenny_leonard',
 19: 'lionel_hutz',
 20: 'lisa_simpson',
 21: 'maggie_simpson',
 22: 'marge_simpson',
 23: 'martin_prince',
 24: 'mayor_quimby',
 25: 'milhouse_van_houten',
 26: 'miss_hoover',
 27: 'moe_szyslak',
 28: 'ned_flanders',
 29: 'nelson_muntz',
 30: 'otto_mann',
 31: 'patty_bouvier',
 32: 'principal_skinner',
 33: 'professor_john_frink',
 34: 'rainier_wolfcastle',
 35: 'ralph_wiggum',
 36: 'selma_bouvier',
 37: 'sideshow_bob',
 38: 'sideshow_mel',
 39: 'snake_jailbird',
 40: 'troy_mcclure',
 41: 'waylon_smithers'}

In [7]:
model = Sequential()

In [8]:
#Using 6 VGG-like blocks, based in the following post: 
#https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(IMG_SIZE, IMG_SIZE, 3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=1024, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
                                  
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(42, activation="softmax"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [11]:
import tensorflow
print(tensorflow.test.gpu_device_name())

/device:GPU:0


In [12]:
EPOCHS = 15
with tensorflow.device('/device:GPU:0'):
    history = model.fit(train_generator, batch_size=64, epochs=EPOCHS)

Epoch 1/15
328/328 [==============================] - 506s 2s/step - loss: 3.2204 - accuracy: 0.1030
Epoch 2/15
328/328 [==============================] - 338s 1s/step - loss: 2.2752 - accuracy: 0.3792
Epoch 3/15
328/328 [==============================] - 339s 1s/step - loss: 1.3279 - accuracy: 0.6507
Epoch 4/15
328/328 [==============================] - 339s 1s/step - loss: 0.9079 - accuracy: 0.7624
Epoch 5/15
328/328 [==============================] - 341s 1s/step - loss: 0.7371 - accuracy: 0.8106
Epoch 6/15
328/328 [==============================] - 346s 1s/step - loss: 0.6223 - accuracy: 0.8419
Epoch 7/15
328/328 [==============================] - 344s 1s/step - loss: 0.5604 - accuracy: 0.8578
Epoch 8/15
328/328 [==============================] - 353s 1s/step - loss: 0.5325 - accuracy: 0.8706
Epoch 9/15
328/328 [==============================] - 349s 1s/step - loss: 0.5362 - accuracy: 0.8737
Epoch 10/15
328/328 [==============================] - 351s 1s/step - loss: 0.5092 - accura

Training process can take between **90** and **120** minutes

In [13]:
model.save('simpsons.h5')